# スケジュール算出プログラム

## 1.ライブラリのインポート
pandasは表の処理

In [ ]:
import random
import datetime
import numpy as np
import pandas as pd
from array import array
import locale                               # ローカライズ用パッケージ

## 2.時間のローカライズ

In [ ]:
locale.setlocale(locale.LC_TIME, "ja_JP")   # 日本にローカライズ（これがないと、英語の曜日が出えてくる）

In [ ]:
def read_excels():

    excel_path = "f_iden_3.xlsm" 
    


    off_day_df  = pd.read_excel(excel_path, sheet_name="yukyu")                         # 休みの希望表
    skill_df    = pd.read_excel(excel_path, sheet_name="skill", header=[0, 1])          # スキル表
    due_date_df = pd.read_excel(excel_path, sheet_name="calendar")                      # 納期表
    start_day:pd.Series = pd.read_excel(excel_path, sheet_name="start")    

    # 空データ(NaN)の置換(0)
    off_day_df.fillna(0, inplace=True)

    due_date_df.fillna(0, inplace=True)

    # データの置換
    off_day_df.replace("◎", 2, inplace=True)                                            # 希望休（◎）を 2 に置換
    due_date_df.replace("☆", 3, inplace=True)                                          # 納期（☆）を 3 に置換

    # 列ごとのデータ型(dtype)を、intに変更しておく
    off_day_df[off_day_df.columns[2:]] = off_day_df[off_day_df.columns[2:]].astype("int64")
    due_date_df[due_date_df.columns[5:]] = due_date_df[due_date_df.columns[5:]].astype("int64")   #スキルグループ→工程による列追加

    # 品番の抽出
    product_num = due_date_df.loc[:, "品番"]

    # 土日の削除
    # データの中から土日の日付を取り出す
    off_days = [date for date in due_date_df.columns[5:] if date.strftime("%a") in "土日"]  

    # 土日の日付を削除する
    off_day_df.drop(columns=off_days, inplace=True)
    due_date_df.drop(columns=off_days, inplace=True)

    return off_day_df, skill_df, due_date_df, product_num , start_day

In [ ]:
off_day_df, skill_df, due_date_df, product_num , start_day = read_excels()

## 3.第1世代関数

In [ ]:
def first_gen(off_day_df: pd.DataFrame, skill_df: pd.DataFrame, due_date_df: pd.DataFrame, product_num: pd.Series):
    
    # 調整後の納期表（納期表のコピー）　
    adjusted_due_date_df = due_date_df.copy()


    # 作業者の割り振り　と　作業者の希望休を調整納期表にコピー
    worker_list = []
    # indexごとにrowを取り出して、作業者をランダムに割り振る
    for index, row in adjusted_due_date_df.iterrows():



        workers: pd.DataFrame = skill_df.loc[:, (["スキルグループ"], [row["スキルグループ"]])] 
        worker = workers.dropna().sample().values[0, 0]

        worker_list.append(worker)
        worker_off_days: pd.DataFrame = off_day_df.loc[off_day_df["設備"].isin([worker]), off_day_df.columns[2:]]                   # 作業者の希望休を抽出


        adjusted_due_date_df.loc[[index], adjusted_due_date_df.columns[5:]] = worker_off_days.values   
 
    # 割り当てたワーカーを adjusted_due_date_df へ「作業者」列として追加（※ここから adjusted_due_date_df が一列増えるので注意）
    adjusted_due_date_df.insert(3, "作業者", worker_list)

   
    # 作業者の希望休に合わせて、納期を調整
    after_index = None                                                                                                              # 後工程の index を保持する
    
    for index, row in adjusted_due_date_df[::-1].iterrows():                                                                        # due_date_df を逆順で行ごとに処理

        # 現工程の納期を抽出
        process: pd.Series = due_date_df.loc[index, due_date_df.columns[5:]]                                                        

        process_due_date = process[process.isin([3])]                                                                               # 現工程の納期
        # print("入力納期:", process_due_date.index[0])
        # print("入力納期process_due_date:", process_due_date)

        if after_index != None:

            after_process_df: pd.DataFrame = adjusted_due_date_df[adjusted_due_date_df["製造番号"] == row["製造番号"]]                               # 同じ製造番号のタスクを取り出す。


            print("[INFO]: 工程:", row["工程"]) 


            if row["工程"] + 1 in after_process_df["工程"].values:

                print("後工程あり")


                after_index = after_process_df[after_process_df["工程"].isin([row["工程"] + 1])].index[0]
                #print("after_index :", after_index)

                # 後工程の納期を抽出
                after_process: pd.Series = adjusted_due_date_df.loc[after_index, adjusted_due_date_df.columns[6:]]                 #スキルグループ→工程による列追加
                after_process_due_date = after_process[after_process.isin([3])]                                                     # 後工程の納期
                #print("後工程の納期:", after_process_due_date.index[0])


                if not process_due_date.index[0] < after_process_due_date.index[0]:

                    # 納期を１営業日早めて調整する（この時点では、有休をチェックしてないので決定しない）
                    before_days:pd.DataFrame = due_date_df.loc[[index], due_date_df.columns[5]:after_process_due_date.index[0]]     # 現在の納期より前のデータを抽出

                    new_due_date = before_days.columns[-2]                                                                          # 納期を１営業日早める
                    #print("後工程を考慮した納期調整:", new_due_date)
                    
                    due_date_df.loc[index, process_due_date.index[0]] = 0                                                           # 現在の納期を納期表から削除
                    due_date_df.loc[index, new_due_date] = process_due_date.values[0]                                               # 新しい納期を納期表に追加

                    process: pd.Seies = due_date_df.loc[index, due_date_df.columns[5:]]                                            

                    process_due_date = process[process.isin([3])]                                                                   # 現工程の納期
                    print("現工程の納期:", process_due_date.index[0])

            else:

                print("後工程なし:")
        
        else:

            print("後工程なし（1つ目のデータ）:")
        
        
        # 希望休に合わせた調整
        # 現在の納期より前のデータを抽出
        before_days: pd.DataFrame = adjusted_due_date_df.loc[[index], adjusted_due_date_df.columns[6]:process_due_date.index[0]] #スキルグループ→工程による列追加


        for new_due_date in before_days.columns[::-1]:
                        
            # 納期を１日早める(-1日)
            #print("納期候補:", new_due_date)

            # その日が希望休でない場合(0 のとき)　→　納期決定　→　ループ終了
            if adjusted_due_date_df.loc[index, new_due_date] == 0:

                # 納期が確定するのはココだけ。valuesで[3]が入るindexと配列指定。values[0]で3指定。
                #print("納期決定:", new_due_date)
                adjusted_due_date_df.loc[index, new_due_date] = process_due_date.values[0]
                break
        
        after_index = index
        #print("-" * 100)              # デバッグ用（ループごとに横線を表示）

    return adjusted_due_date_df


In [ ]:
#受取
adjusted_due_date_df = first_gen(off_day_df, skill_df, due_date_df, product_num)

In [ ]:
#第1世代

## ※　データのルール
製造番号は必ず連番

In [ ]:
adjusted_due_date_df[adjusted_due_date_df["製造番号"] == 1].iloc[:, :15]

## 4.納期をランダムで振り直し（再帰処理）
※　この処理中は希望休を考慮しない


### 開始日

In [ ]:
start_date = datetime.datetime(2023, 6, 2)

### 納期のランダム振り分け関数

In [ ]:
def generate_random_dates(
        start_date: datetime.datetime, 
        end_date: datetime.datetime):
    
    start_date = due_date_df.loc[:, start_date:].columns[1]

    working_days = due_date_df.loc[:, start_date:end_date].columns
    #print(working_days)

    random_num = random.randint(0, working_days.shape[0]-1)
    random_date = working_days[random_num]
    #print(random_date)
    #print("-" * 50, "納期ランダム割り振り完了","-" * 50)

    return random_date

### 同じ製造番号のタスクをランダムにリスケする関数（これが再帰関数：関数内の処理に自分自身の呼び出しがある）

In [ ]:
def random_due_date(
        start_date: datetime.datetime,
        process_df: pd.DataFrame,
        process_num: int):
    
    #print("process_num :", process_num, "-" * 100)
    #print('process_df',process_df.loc[[process_df.index[process_num]], process_df.columns[:5]])

    process: pd.Series = process_df.loc[process_df.index[process_num], process_df.columns[5:]]              # 現工程の納期表データ
    deadline = process[process.isin([3])]                                                                   # 現工程の納期
    deadline_date: datetime.datetime = deadline.index[0]                                                    # その工程のデッドライン
    #print("process_deadline :", deadline_date)


    if process_num != 0:

        start_date = random_due_date(start_date, process_df.loc[process_df.index[:-1], :], process_num-1)   #（再帰処理）

    new_due_date = generate_random_dates(start_date, deadline_date)                                         # 開始日～デッドライン　の期間からランダムに日付を選択

    due_date_df.loc[process_df.index[process_num], deadline.index[0]] = 0                                   # 現在の納期を納期表から削除
    due_date_df.loc[process_df.index[process_num], new_due_date] = deadline.values[0]                       

    return new_due_date
                     


### タスクを製造番号ごとに処理する関数

In [ ]:
def change_due_date():
    
    serial_number = due_date_df.loc[:, "製造番号"].max()
    #print('serial_number',serial_number)

    # 製造番号ごとに繰り返す
    for i in range(1, serial_number + 1):

        # 同じ製造番号のタスクを取り出す。
        process_df = adjusted_due_date_df[adjusted_due_date_df["製造番号"] == i]

        #print('process_df.shape',process_df.shape)
        #print('process_df.loc[:, process_df.columns].isin([3])')



        random_due_date(start_date, process_df, process_df.shape[0]-1)                                          # 製造番号ごとに納期のランダム割り当て（再帰関数）

        #print('タスク',process_df)
        #print('process_df.shape[0]',process_df.shape[0])
        #print("*" * 200)

### ※　この時点では、希望休を考慮した納期になっていない為、first_gen()関数を使ってもう一度希望休に沿った調整をする。



In [ ]:
# 呼び出し
change_due_date()

### ※　この時点では、希望休を考慮した納期になっていない為、first_gen()関数を使ってもう一度希望休に沿った調整をする。

In [ ]:
adjusted_due_date_df = first_gen(off_day_df, skill_df, due_date_df, product_num)

#### これでランダムな納期かつ希望休の調整ができる

##### 希望休表

##### 希望休調整前(ランダム)

##### 希望休調整後

In [ ]:
adjusted_due_date_df_copy= adjusted_due_date_df.copy()
adjusted_due_date_df_copy=adjusted_due_date_df_copy.iloc[:,5:]
adjusted_due_date_df_copy.reset_index(inplace=True,drop=True)
#行と列の数を取り出す
sh3=adjusted_due_date_df_copy.shape
adjusted_due_date_df_copy.columns=range(sh3[1])


In [ ]:
#45変換
def pre_evalution(adjusted_due_date_df):
    adjusted_due_date_df2=adjusted_due_date_df.iloc[:,6:] #スキルグループ→工程による列追加

    return adjusted_due_date_df2

In [ ]:
#受取
adjusted_due_date_df2= pre_evalution(adjusted_due_date_df)

## 5.評価方法　

In [ ]:
##(1)工程ができるだけ長くならないこと
##(2)前倒し生産方式の場合、できるだけ開始日に近いこと、ジャストインタイム方式の場合、できるだけ納期に近いこと。
#評価方法
def evalution_function(adjusted_due_date_df2):
    
    
##(3)前倒し生産方式の場合、できるだけ開始日に近いこと、ジャストインタイム方式の場合、できるだけ納期に近いこと。          
#秒換算
# ’×－1 
    score3=[]  
    fix_day3= []
    tdos_list3=[]
    adjusted_due_date_df_copy= adjusted_due_date_df2.copy()
    adjusted_due_date_df_copy.reset_index(inplace=True,drop=True)
    #行と列の数を取り出す
    sh3=adjusted_due_date_df_copy.shape
    adjusted_due_date_df_copy.columns=range(sh3[1])
    #print('sh3',sh3)
    #print('sh3[1]',sh3[1])
               
        # 納期を１日早める(-1日)

    for k in range(len(adjusted_due_date_df_copy)):
        for v in range(len(adjusted_due_date_df_copy.columns)):
            if adjusted_due_date_df_copy.iloc[k,v]==3:
                fix_day3.append(adjusted_due_date_df.columns[v+5])
                #fix_day.append(adjusted_due_date_df_copy.columns[v])
        # その日が希望休でない場合(0 のとfixき)　→　納期決定　→　ループ終了

    # print('fix_day3',fix_day3)
    # print(type(fix_day3))
    # print('start_date',start_date)
    # print(type(start_date))
    tdos3= '0000-00-00 00:00:00'
    tm3 : datetime.datetime

    for tm3 in fix_day3:
        # print('tm',tm3)
        # print(type(tm3))
        ttdelta3= tm3- start_date 
        #print('ttdelta3',ttdelta3)
        tdos3= ttdelta3.total_seconds()
        #print('tdos3',tdos3)
        tdos3=int(tdos3)
        tdos_list3.append(tdos3)
    #print('tdos_list3',tdos_list3)
    score3=sum(tdos_list3)*(-1)
    #print('score3',score3)

#追加
##(4)工程ができるだけ長くならないこと
#工程間1日ずれ（品番＝＝品番）len1=（日程（工程ｎ＋１）-（日程（工程ｎ）>0なら高得点
    # ’×－1 
    score4=[]  
    fix_day4= []
    fix_day41= []
    fix_day42= []
    tdos_list4=[]
    ttdos_list4=[]
    ttdos_list40=[]
    delta4=[]
    delta40=[]
    tdos4=[]

##タスクを製造番号ごとに処理する関数コピー
    serial_number2 = due_date_df.loc[:, "製造番号"].max()
    #print('serial_number2',serial_number2)

    # 製造番号ごとに繰り返す
    for i2 in range(1, serial_number2 + 1):

        # 同じ製造番号のタスクを取り出す。
        process_df2 = adjusted_due_date_df[adjusted_due_date_df["製造番号"] == i2]
        #列の表示数の上限を撤廃
        pd.set_option('display.max_columns', None)
        # print('process_df2',process_df2)
        # print('process_df2.shape[0]',process_df2.shape[0])
        # print('process_df2.shape[1]',process_df2.shape[1])
        # print('process_df2.columns',process_df2.columns)
        # print('process_df2',process_df2)
        for k2 in range(0,len(process_df2)):
            for v2 in range(6,len(process_df2.columns)- 5):
                if process_df2.iloc[k2,v2]==3:
                    fix_day4.append(process_df2.columns[v2])
        #追加1        
        h2=len(process_df2)
        #print('h2',h2)
        for g2 in range(h2 -1, 0, -1):
       
            #print('g2',g2)

            delta40=(fix_day4[g2]- fix_day4[g2 -1])

            #print('delta40',delta40)
            delta4=delta40.total_seconds()
            
            #print('delta4',delta4)
            delta4=int(delta4)
            #追加
            
            if delta4== 86400:
                tdos4= delta4- 100000
                tdos_list4.append(tdos4)
            elif delta4< 0:
                tdos4= delta4+ 100000
                tdos_list4.append(tdos4)
            elif delta4== 86400*2:
                tdos4= delta4-50000
                tdos_list4.append(tdos4)
            else:
                tdos4= delta4
                tdos_list4.append(tdos4)
            #print('tdos_list4',tdos_list4)
            ttdos_list40=sum(tdos_list4)
            #print('ttdos_list40',ttdos_list40)
        ttdos_list4.append(ttdos_list40)
        #print('ttdos_list4',ttdos_list4)
    score4=sum(ttdos_list4)
    #print('score4',score4)
    score=  score3+score4
    return score


In [ ]:
#受取
score= evalution_function(adjusted_due_date_df2)

## 6.遺伝的アルゴリズム

In [ ]:
##62遺伝的アルゴリズム


def crossover(ep,sd,p1,p2):
    #1か月の日数
    #days=len(p1.columns)
    days= len(off_day_df.columns) - 2
    #1次元化
    p1=np.array(p1).flatten()
    p2=np.array(p2).flatten()

    #print(p1)
    #print(p2)
    #子の変数
    ch1=[]
    ch2=[]
    for p1_,p2_ in zip(p1,p2):
        if ep > random.random():
            ch1.append(p1_)
            ch2.append(p2_)
        else:
            ch1.append(p2_)
            ch2.append(p1_)

    #突然変異
    ch1=mutation(sd, np.array(ch1).flatten())
    ch2=mutation(sd, np.array(ch2).flatten())
    
    #pandasに変換
    ch1=pd.DataFrame(ch1.reshape([int(len(ch1)/days), days]))
    ch2=pd.DataFrame(ch2.reshape([int(len(ch2)/days), days]))
    #列名の変更
    ch1.columns=[i+1 for i in range(len(ch1.columns))]
    ch2.columns=[i+1 for i in range(len(ch2.columns))]
    return ch1,ch2


#突然変異
def mutation(sd2, ch):
    if sd2 > random.random():

        rand = np.random.permutation(list(range(len(ch))))

        #遺伝子の10％を変異させる
        #並び換えた先頭10％
        rand=rand[:int(len(ch)//10)]
        for i in rand:
            #1なら0,0なら1
            if ch[i]==3:
                ch[i]==0

            if ch[i]==0:
                ch[i]==3
                #ch2も同じことをする。

    return ch


## 7.全体

In [ ]:
##72全体
#遺伝的アルゴリズム
#エクセル読み込み
off_day_df, skill_df, due_date_df, product_num , start_day =read_excels()
#親の保存
parent=[]
#for i in range(100):
#3世代へ変更 停止用
for i in range(3):
    #第1世代
    adjusted_due_date_df = first_gen(off_day_df, skill_df, due_date_df, product_num)
    #休日数の修正

    change_due_date()
    adjusted_due_date_df = first_gen(off_day_df, skill_df, due_date_df, product_num)


    #評価
    adjusted_due_date_df2= pre_evalution(adjusted_due_date_df)
    score= evalution_function(adjusted_due_date_df2)
    #第1世代を格納
    parent.append([score, adjusted_due_date_df2.values]) #

#print('parent',parent)



#上位個体数
elite_length=20
#世代数
gene_length=50

#一様交叉確立
ep=0.5
#突然変異確立
sd=0.05


for i in range(gene_length):
    #点数で並び換え
    parent = sorted(parent, key=lambda x: -x[0])
    #上位個体を選別
    parent=parent[:elite_length]
    #各世代
    print('第'+str(i+1)+'世代')

    #最高得点の更新
    if i ==0:
        top=parent[0]
            #各世代の最高得点の表示
        print(top[0])
        print(np.array(top[1]))
    else:
        if top[0]>parent[0][0]:
            parent.append(top)
            #各世代の最高得点の表示
            #print(top[0])
            #print(np.array(top[1]))
        else:
            top=parent[0]
            #各世代の最高得点の表示
            #print(parent[0][0])
            #print(np.array(parent[0][1]))

    #子世代
    children=[]
    #遺伝子操作
    for k1,v1 in enumerate(parent):
        for k2,v2 in enumerate(parent):
            if k1<k2:
                #一様交叉
                # print(ep)
                # print(sd)
                # print(v1[1])
                # print(v2[1])
                ch,ch2=crossover(ep,sd,v1[1],v2[1])
                #休日数変更
                change_due_date()
                adjusted_due_date_df = first_gen(off_day_df, skill_df, due_date_df, product_num)
                #評価
                score1=evalution_function(ch)
                score2=evalution_function(ch2)
                #子孫を格納
                children.append([score1,ch])
                children.append([score2,ch2])

    #子を親にコピー
    parent=children.copy()

#最強個体の保存
x=top[1].replace(3,'☆').replace(2,'◎').replace(0,'')
x.to_excel('f_shiftf_4237.xlsx')
